In [1]:
import pandas as pd
import numpy as np
from data_cleaning import clean_flights_df, clean_passengers_df, avg_passengers, clean_fuel_df, avg_carrier_arr_delay, avg_fuel_use, avg_taxi_time

In [2]:
flights_df = clean_flights_df('data/flights_sample.csv')
flights_df.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,...,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,year,month,day_of_month,day_of_week,dep_hour
0,2019-07-17,UA,UA,613,UA,N454UA,613,14771,SFO,"San Francisco, CA",...,0.0,0.0,0.0,0.0,0.0,2019,7,17,2,6
1,2018-06-09,WN,WN,5610,WN,N712SW,5610,10821,BWI,"Baltimore, MD",...,0.0,7.0,0.0,0.0,0.0,2018,6,9,5,13
2,2019-11-06,B6,B6,737,B6,N705JB,737,10529,BDL,"Hartford, CT",...,0.0,0.0,0.0,0.0,0.0,2019,11,6,2,6
3,2019-12-24,WN,WN,6221,WN,N8503A,6221,12889,LAS,"Las Vegas, NV",...,0.0,0.0,0.0,0.0,0.0,2019,12,24,1,9
4,2018-01-14,WN,WN,2663,WN,N446WN,2663,13204,MCO,"Orlando, FL",...,0.0,0.0,0.0,0.0,0.0,2018,1,14,6,20


In [3]:
flights_df.shape

(4681523, 42)

In [4]:
flights_df = flights_df.sample(frac=.5, random_state=58)
flights_df.shape

(2340762, 42)

In [5]:
passengers_df = clean_passengers_df('data/passengers.csv')
fuel_df = clean_fuel_df('data/fuel_consumption.csv')

### Feature Selection/Engineering

In [6]:
y = flights_df['dep_delay']
X = flights_df.drop('dep_delay', axis=1)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=58)

In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1638533 entries, 1336231 to 1911158
Data columns (total 41 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   fl_date              1638533 non-null  datetime64[ns]
 1   mkt_unique_carrier   1638533 non-null  object        
 2   branded_code_share   1638533 non-null  object        
 3   mkt_carrier_fl_num   1638533 non-null  int64         
 4   op_unique_carrier    1638533 non-null  object        
 5   tail_num             1638533 non-null  object        
 6   op_carrier_fl_num    1638533 non-null  int64         
 7   origin_airport_id    1638533 non-null  int64         
 8   origin               1638533 non-null  object        
 9   origin_city_name     1638533 non-null  object        
 10  dest_airport_id      1638533 non-null  int64         
 11  dest                 1638533 non-null  object        
 12  dest_city_name       1638533 non-null  object        
 

In [11]:
# create average taxi in and out columns.
taxi_dep_mean_dict = round(X_train.groupby(X_train['dep_time']//100)['taxi_out'].mean(), 2).to_dict()
taxi_arr_mean_dict = round(X_train.groupby(X_train['arr_time']//100)['taxi_in'].mean(), 2).to_dict()

X_train['arr_hour'] = X_train['arr_time'] // 100
X_train['taxi_dep_mean'] = X_train['dep_hour'].map(taxi_dep_mean_dict)
X_train['taxi_arr_mean'] = X_train['arr_hour'].map(taxi_arr_mean_dict)
X_train.drop('arr_hour', axis=1, inplace=True)

In [13]:
# create avg_carrier_delay column
avg_carrier_arr_delay_dict = round(X_train.groupby('op_unique_carrier')['arr_delay'].mean(), 2).to_dict()
X_train['avg_carrier_arr_delay'] = flights_df['op_unique_carrier'].map(avg_carrier_arr_delay_dict)

In [14]:
# add engineered features to training data
X_train = avg_passengers(X_train, passengers_df)
X_train = avg_fuel_use(X_train, fuel_df)
X_train.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,...,month,day_of_month,day_of_week,dep_hour,taxi_dep_mean,taxi_arr_mean,avg_carrier_arr_delay,monthly_avg_passengers,avg_monthly_fuel_gallons,avg_monthly_fuel_cost
1336231,2019-10-05,DL,DL_CODESHARE,4644,OO,N254SY,4644,12954,LGB,"Long Beach, CA",...,10,5,5,10,17.78,7.26,7.29,5438.0,284918524.0,578736044.0
3364978,2019-07-27,DL,DL_CODESHARE,5101,9E,N134EV,5101,11042,CLE,"Cleveland, OH",...,7,27,5,14,16.83,7.20,3.24,1456.0,338521162.0,631140692.0
1654928,2019-07-17,G4,G4,168,G4,323NV,168,12889,LAS,"Las Vegas, NV",...,7,17,2,14,16.83,8.43,8.99,1615.0,18978430.0,37629674.0
1351977,2018-05-26,DL,DL,1631,DL,N717TW,1631,12892,LAX,"Los Angeles, CA",...,5,26,5,8,19.42,7.70,0.23,4421.0,303232966.0,598183733.0
1178290,2018-10-13,WN,WN,5017,WN,N8501V,5017,11292,DEN,"Denver, CO",...,10,13,5,10,17.78,7.38,3.63,2705.0,166445147.0,317502832.0


In [38]:
X_train.isna().sum()

fl_date                     0
mkt_unique_carrier          0
branded_code_share          0
mkt_carrier_fl_num          0
op_unique_carrier           0
tail_num                    0
op_carrier_fl_num           0
origin_airport_id           0
origin                      0
origin_city_name            0
dest_airport_id             0
dest                        0
dest_city_name              0
crs_dep_time                0
dep_time                    0
taxi_out                    0
wheels_off                  0
wheels_on                   0
taxi_in                     0
crs_arr_time                0
arr_time                    0
arr_delay                   0
cancelled                   0
crs_elapsed_time            0
actual_elapsed_time         0
air_time                    0
flights                     0
distance                    0
carrier_delay               0
weather_delay               0
nas_delay                   0
security_delay              0
late_aircraft_delay         0
first_dep_

In [37]:
mean_passengers = X_train['monthly_avg_passengers'].mean()
X_train.fillna(value=mean_passengers, inplace=True)

In [40]:

# Calculate map mean taxi time per hour from training data to test data.
X_test['taxi_dep_mean'] = X_test['dep_hour'].map(taxi_dep_mean_dict)

X_test['arr_hour'] = X_test['crs_arr_time'] // 100
X_test['taxi_arr_mean'] = X_test['arr_hour'].map(taxi_arr_mean_dict)
X_test.drop('arr_hour', axis=1, inplace=True)

X_test.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,...,first_dep_time,total_add_gtime,longest_add_gtime,year,month,day_of_month,day_of_week,dep_hour,taxi_dep_mean_time,taxi_arr_mean_time
3944851,2018-08-22,DL,DL_CODESHARE,4775,OO,N604SK,4775,14869,SLC,"Salt Lake City, UT",...,0.0,0.0,0.0,2018,8,22,2,15,17.63,7.70
3459129,2018-07-15,WN,WN,4408,WN,N8571Z,4408,14107,PHX,"Phoenix, AZ",...,0.0,0.0,0.0,2018,7,15,6,8,19.42,7.26
4457285,2019-07-01,WN,WN,1970,WN,N8575Z,1970,15016,STL,"St. Louis, MO",...,0.0,0.0,0.0,2019,7,1,0,16,17.28,7.70
17946,2019-04-16,AA,AA_CODESHARE,5415,OH,N248PS,5415,10257,ALB,"Albany, NY",...,1427.0,24.0,24.0,2019,4,16,1,14,16.83,7.38
2506578,2019-03-25,NK,NK,754,NK,N620NK,754,13204,MCO,"Orlando, FL",...,0.0,0.0,0.0,2019,3,25,0,21,17.54,6.87


In [41]:
X_test['avg_carrier_arr_delay'] = X_test['op_unique_carrier'].map(avg_carrier_arr_delay_dict)

In [42]:
X_test = avg_passengers(X_test, passengers_df)
X_test = avg_fuel_use(X_test, fuel_df)

In [43]:
X_test.isna().sum()

fl_date                     0
mkt_unique_carrier          0
branded_code_share          0
mkt_carrier_fl_num          0
op_unique_carrier           0
tail_num                    0
op_carrier_fl_num           0
origin_airport_id           0
origin                      0
origin_city_name            0
dest_airport_id             0
dest                        0
dest_city_name              0
crs_dep_time                0
dep_time                    0
taxi_out                    0
wheels_off                  0
wheels_on                   0
taxi_in                     0
crs_arr_time                0
arr_time                    0
arr_delay                   0
cancelled                   0
crs_elapsed_time            0
actual_elapsed_time         0
air_time                    0
flights                     0
distance                    0
carrier_delay               0
weather_delay               0
nas_delay                   0
security_delay              0
late_aircraft_delay         0
first_dep_

In [44]:
mean_passengers = X_test['monthly_avg_passengers'].mean()
X_test.fillna(value=mean_passengers, inplace=True)

In [46]:
del passengers_df
del fuel_df

In [47]:

col_to_remove = ['fl_date','air_time','dep_hour', 'tail_num','mkt_unique_carrier', 'branded_code_share', 'mkt_carrier_fl_num', 'op_carrier_fl_num', 'origin_airport_id', 'origin_city_name', 'dest_airport_id', 'dest_city_name', 'dep_time', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in', 'arr_time', 'arr_delay', 'cancelled', 'actual_elapsed_time', 'flights', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'first_dep_time', 'total_add_gtime', 'longest_add_gtime',]

In [48]:
X_train = X_train.drop(columns=col_to_remove)
X_train.head()

,op_unique_carrier,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,year,month,day_of_month,day_of_week,taxi_dep_mean,taxi_arr_mean,avg_carrier_arr_delay,monthly_avg_passengers,avg_monthly_fuel_gallons,avg_monthly_fuel_cost
1336231,OO,LGB,SLC,1010,1305,115.0,588,2019,10,5,5,17.78,7.26,7.29,5438.0,284918524.0,578736044.0
3364978,9E,CLE,JFK,1445,1630,105.0,425,2019,7,27,5,16.83,7.20,3.24,1456.0,338521162.0,631140692.0
1654928,G4,LAS,SHV,1443,1942,179.0,1243,2019,7,17,2,16.83,8.43,8.99,1615.0,18978430.0,37629674.0
1351977,DL,LAX,DCA,825,1633,308.0,2311,2018,5,26,5,19.42,7.70,0.23,4421.0,303232966.0,598183733.0
1178290,WN,DEN,RDU,1055,1605,190.0,1436,2018,10,13,5,17.78,7.38,3.63,2705.0,166445147.0,317502832.0


In [49]:
X_test = X_test.drop(columns=col_to_remove)
X_test.head()

,op_unique_carrier,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,year,month,day_of_month,day_of_week,taxi_dep_mean_time,taxi_arr_mean_time,avg_carrier_arr_delay,monthly_avg_passengers,avg_monthly_fuel_gallons,avg_monthly_fuel_cost
3944851,OO,SLC,FCA,1520,1706,106.0,531,2018,8,22,2,17.63,7.70,7.29,1703.0,337147231.0,612541493.0
3459129,WN,PHX,MKE,820,1340,200.0,1460,2018,7,15,6,19.42,7.26,3.63,3209.0,185130079.0,351005250.0
4457285,WN,STL,LAS,1620,1750,210.0,1371,2019,7,1,0,17.28,7.70,3.63,5268.0,185130079.0,351005250.0
17946,OH,ALB,DCA,1430,1559,89.0,318,2019,4,16,1,16.83,7.38,7.35,1306.0,275614616.0,471106534.0
2506578,NK,MCO,PHL,2130,2356,146.0,861,2019,3,25,0,17.54,6.87,5.17,5834.0,28398402.0,56354459.0


In [50]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(1638533, 17) (702229, 17)
(1638533,) (702229,)


Data Split and engineered features added at this point.

## Attempt at XGBoost

In [56]:
cats = X_train.select_dtypes(exclude=np.number).columns.tolist()

for col in cats:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')


In [57]:
import xgboost as xgb

dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [58]:
params = {'objective':'reg:squarederror', 'tree_method':'hist'}
n=1000
evals=[(dtrain_reg, 'train'), (dtest_reg, 'validation')]

model = xgb.train(
    params=params,
    dtrain=dtrain_reg,
    num_boost_round=n,
    evals=evals,
    verbose_eval=25,
    early_stopping_rounds=50
)


[0]	train-rmse:44.61843	validation-rmse:48.20584
[25]	train-rmse:22.47484	validation-rmse:48.06710
[50]	train-rmse:19.52600	validation-rmse:48.32661
[57]	train-rmse:19.07251	validation-rmse:48.36487


In [59]:
y_pred = model.predict(dtest_reg)

In [60]:
from sklearn.metrics import r2_score, mean_squared_error

print(mean_squared_error(y_test, y_pred, squared=False))
print(r2_score(y_test, y_pred))

48.36625412661141
-0.03070843591227601


In [63]:
X_train.columns

Index(['op_unique_carrier', 'origin', 'dest', 'crs_dep_time', 'crs_arr_time',
       'crs_elapsed_time', 'distance', 'year', 'month', 'day_of_month',
       'day_of_week', 'taxi_dep_mean', 'taxi_arr_mean',
       'avg_carrier_arr_delay', 'monthly_avg_passengers',
       'avg_monthly_fuel_gallons', 'avg_monthly_fuel_cost'],
      dtype='object')

In [61]:
import pickle
filename = 'Saved_models/Ben_XGBmodel1.sav'
pickle.dump(model, open(filename, 'wb'))

## Random Forest Regressor

In [64]:
cats = X_train.select_dtypes(exclude=np.number).columns.tolist()
cats

['op_unique_carrier', 'origin', 'dest']

In [65]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
X_train.reset_index(inplace=True)
le = LabelEncoder()
X_cats = X_train[cats].apply(le.fit_transform)
X_cats.head()

,op_unique_carrier,origin,dest
0,19,206,332
1,0,72,185
2,12,194,327
3,8,196,93
4,24,94,297


In [66]:
enc = OneHotEncoder()
enc.fit(X_cats)

onehotlabels = enc.transform(X_cats)

# onehotlabels

In [67]:
encoded_df = pd.DataFrame.sparse.from_spmatrix(onehotlabels)

In [68]:
print('encoded: ', encoded_df.shape)
print('training set:', X_train.shape)
print('y:', y_train.shape)

encoded:  (1638533, 778)
training set: (1638533, 18)
y: (1638533,)


In [69]:
X_train_encoded = pd.concat([X_train, encoded_df], axis=1, ignore_index=True)
X_train_encoded.shape

(1638533, 796)

In [70]:
X_train_encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,786,787,788,789,790,791,792,793,794,795
0,1336231,OO,LGB,SLC,1010,1305,115.0,588,2019,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3364978,9E,CLE,JFK,1445,1630,105.0,425,2019,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1654928,G4,LAS,SHV,1443,1942,179.0,1243,2019,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1351977,DL,LAX,DCA,825,1633,308.0,2311,2018,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1178290,WN,DEN,RDU,1055,1605,190.0,1436,2018,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
X_train_encoded.drop(columns=[0, 1, 2, 3], inplace=True)

In [72]:
print(X_train_encoded.shape, y_train.shape)

(1638533, 792) (1638533,)


In [76]:
X_train_encoded.head()

,4,5,6,7,8,9,10,11,12,13,...,786,787,788,789,790,791,792,793,794,795
0,1010,1305,115.0,588,2019,10,5,5,17.78,7.26,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1445,1630,105.0,425,2019,7,27,5,16.83,7.20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1443,1942,179.0,1243,2019,7,17,2,16.83,8.43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,825,1633,308.0,2311,2018,5,26,5,19.42,7.70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1055,1605,190.0,1436,2018,10,13,5,17.78,7.38,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
from sklearn.ensemble import RandomForestRegressor

rfr_model = RandomForestRegressor()


In [80]:

rfr_model.fit(X_train_encoded, y_train)

c:\Users\HP USER\anaconda3\envs\lhl_data\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


MemoryError: Unable to allocate 9.67 GiB for an array with shape (792, 1638533) and data type float64

In [ ]:
# encode test data
X_test.reset_index(inplace=True)
le = LabelEncoder()
X_cats = X_test[cats].apply(le.fit_transform)

enc = OneHotEncoder()
enc.fit(X_cats)

onehotlabels = enc.transform(X_cats)
encoded_df = pd.DataFrame.sparse.from_spmatrix(onehotlabels)
print('encoded: ', encoded_df.shape)
print('training set:', X_test.shape)
print('y:', y_test.shape)

In [ ]:
y_pred = rfr_model.predict(X_test)

## AdaBoost Model

In [81]:
from sklearn.ensemble import AdaBoostRegressor

abr = AdaBoostRegressor()

abr.fit(X_train_encoded, y_train)

c:\Users\HP USER\anaconda3\envs\lhl_data\lib\site-packages\sklearn\utils\validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


MemoryError: Unable to allocate 9.67 GiB for an array with shape (792, 1638533) and data type float64